In [1]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
publico_path = '../extra/notícias/news_sample_1to1000_updated.csv'
observador_path = '../extra/notícias/noticias_observador_fev_a_julho.csv'

In [3]:
df_publico = pd.read_csv(publico_path, index_col='Unnamed: 0')
df_observador = pd.read_csv(observador_path, index_col='Unnamed: 0')

## Data Cleaning

### Notícias Público 

In [4]:
df_publico.head()

,titulo,descricao,titulo_Mobile,subtitulo,rubricTag,rubrica,tipo,palavraChave,tipoLayout,isOpinion,...,isClubeP,isItemOpinion,isMultipleRubric,socialTitle,isLongForm,isHeadline,wordCount,extra4,partners,props
0,Linha de mil milhões para micro e pequenas emp...,Ministério da Economia diz que a formalização ...,NaN,NaN,NaN,Covid-19,NOTICIA,NaN,MULTIMEDIA_NORMAL,False,...,False,False,False,Covid-19. Linha de mil milhões para micro e pe...,False,False,349,NaN,NaN,NaN
1,Covid-19: Twitter retira vídeo e mensagens de ...,"Segundo o jornal <i>The Washington Post</i>, o...",NaN,NaN,NaN,Coronavírus,NOTICIA,NaN,MULTIMEDIA_NORMAL,False,...,False,False,False,Coronavírus. Covid-19: Twitter retira vídeo e ...,False,False,358,NaN,NaN,NaN
2,Tecnologia portuguesa elimina o novo coronavír...,"A tecnologia pioneira AT MicroProtect, desenvo...",NaN,NaN,NaN,Coronavírus,NOTICIA,NaN,MULTIMEDIA_NORMAL,False,...,False,False,False,Coronavírus. Tecnologia portuguesa elimina o n...,False,False,740,NaN,NaN,NaN
3,"“Não somos super-homens, somos humanos”: os re...",Há de tudo: a algumas pessoas a covid-19 não p...,NaN,NaN,NaN,Coronavírus,NOTICIA,NaN,LONGFORM_NORMAL,False,...,False,False,False,"Coronavírus. “Não somos super-homens, somos hu...",True,False,2108,NOTIFIED,NaN,NaN
4,Dez milhões de euros para as instituições part...,"Da aquisição de máscaras, luvas e viseiras à c...",NaN,NaN,NaN,Covid-19,NOTICIA,NaN,MULTIMEDIA_NORMAL,False,...,False,False,False,Covid-19. Dez milhões de euros para as institu...,False,False,330,NaN,NaN,NaN


In [5]:
df_publico.columns

Index(['titulo', 'descricao', 'titulo_Mobile', 'subtitulo', 'rubricTag',
       'rubrica', 'tipo', 'palavraChave', 'tipoLayout', 'isOpinion',
       'prioridade', 'isPreview', 'aoMinuto', 'isFeature', 'dataActualizacao',
       'data', 'autores_id', 'autores_nome', 'autores_localizacao',
       'autores_profissaoActual', 'autores_profissaoNaAltura',
       'autores_isExternal', 'autores_contribuicao', 'autores_tipo', 'tags_id',
       'tags_nome', 'tags_isPrincipal', 'tags_isTimeLine',
       'tags_forcaConteudoAberto', 'tags_isForForuns', 'tags_isUsedInForuns',
       'isSponsorContent', 'isClubeP', 'isItemOpinion', 'isMultipleRubric',
       'socialTitle', 'isLongForm', 'isHeadline', 'wordCount', 'extra4',
       'partners', 'props'],
      dtype='object')

In [6]:
df_publico.palavraChave.isnull().sum()

9670

In [7]:
df_publico.tipo.value_counts()

NOTICIA       9390
EXTERNOS        23
INFOGRAFIA      13
QUIZZ            2
Name: tipo, dtype: int64

In [8]:
# Removendo "tipo" quizz
df_publico = df_publico[df_publico["tipo"]!="QUIZZ"]

In [9]:
# colunas essenciais
df_publico = df_publico[["data", "titulo","descricao"]]

In [10]:
# data para datetime
df_publico.dtypes

data         object
titulo       object
descricao    object
dtype: object

In [19]:
df_publico["data"] = pd.to_datetime(df_publico["data"]).dt.normalize()

In [20]:
# select rows between two dates: 1st of Feb to 30th of June
# Feb because observador only starts on 1st of Feb
start_date = "2020-02-01"
end_date = "2020-06-30"

df_publico = df_publico.loc[(df_publico["data"] >= start_date) & (df_publico["data"] <= end_date)]

In [21]:
df_publico.sort_values(by="data", inplace=True)

In [22]:
df_publico.head()

,data,titulo,descricao
9590,2020-02-01,Caso suspeito no Porto não se confirma. Avião ...,Homem que chegou da China a 22 de Janeiro foi ...
9589,2020-02-01,Mundo fecha as portas a viajantes da China com...,Apesar de ir contra as recomendações da Organi...
9588,2020-02-01,Caso suspeito de coronavírus evidencia problem...,Dono de fábrica por onde passou italiano suspe...
9583,2020-02-02,Vinte cidadãos repatriados de Wuhan ficam em q...,"Governo decidiu, por agora, que os 18 portugue..."
9587,2020-02-02,Coronavírus: a China está a esforçar-se para e...,À medida que o surto de coronavírus sofria um ...


In [23]:
df_publico.shape

(8779, 3)

In [24]:
# duplicated entries
df_publico[df_publico.duplicated()]

,data,titulo,descricao
8559,2020-03-17,Coronavírus: o cisne negro da modernidade\r\n,Era bom que depois de atravessarmos o perigo d...
7109,2020-03-30,Os desafios: evitar o covid-19 e a maior reces...,Medidas do Governo parecem proteger parte da e...
6729,2020-04-02,"Para muitos idosos, a farmácia é hoje um porto...","Testemunho de Sílvia Cid, directora técnica de..."
5869,2020-04-10,A Academia do Porto está a combater a covid-19...,Iniciativa em rede juntou a Universidade e o I...
3510,2020-05-07,Confinamento em casa separado da família? Em 9...,"A taxa de sobrelotação da habitação melhorou, ..."
1690,2020-06-05,A nadar e a puxar pesos outra vez: reabriram a...,Com menos gente e com tempo limite de utilizaç...
1519,2020-06-10,A história de um privilegiado num desporto em ...,"Pedro Cruz tem, no andebol português, uma posi..."


In [ ]:
df_publico.drop_duplicates(keep="first", inplace = True)

### Notícias Observador

In [16]:
df_observador.head()

,Data,Titulo,Desc,Links
0,2020-02-01,Coronavírus. Portugueses ficam 14 dias ...,Os 18 portugueses vindos de Wuhan já aterraram...,https://observador.pt/2020/02/01/aviao-que-vai...
1,2020-02-01,"Wuhan Coronavírus: afinal, qual o risco?",Os coronavírus é uma família de vírus com prov...,https://observador.pt/opiniao/2019-ncov-ou-wuh...
2,2020-02-01,Como o coronavírus afeta fabricantes de...,"Com mais habitantes do que Portugal, Wuhan é u...",https://observador.pt/2020/01/31/como-o-corona...
3,2020-02-01,Centenas são repatriados a partir de Wuhan,Vários países já estão a repatriar os seus cid...,https://observador.pt/programas/atualidade/cen...
4,2020-02-01,Vídeo mostra a origem do coronavírus?,Vídeo que alegadamente mostra um mercado na Ch...,https://observador.pt/factchecks/fact-check-vi...


In [17]:
df_observador.drop_duplicates(subset = "Links",keep = "first", inplace = True)

In [ ]:
# changing column names
df_observador.rename(columns = {"Data": "data","Titulo":"titulo", "Desc": "descricao", "Links":"links"}, inplace=True)

In [ ]:
# selecting dates
start_date = "2020-02-01"
end_date = "2020-06-30"

df_observador = df_observador.loc[(df_observador["data"] >= start_date) & (df_observador["data"] <= end_date)]



In [18]:
print("Observador shape:  ", df_observador.shape)
print("Público shape:   ", df_publico.shape)


Observador shape:   (12543, 4)
Público shape:    (9668, 3)


# VISUALIZAÇAO

#### Evolução de notícias - Público

In [29]:
import plotly.express as px

In [45]:
# groupby? 
publico_daily_news = df_publico.groupby(["data"]).count()
publico_daily_news.rename(columns={"titulo":"Número de Notícias Diárias"}, inplace=True)

In [42]:
publico_daily_news.columns

Index(['Número de Notícias Diárias', 'descricao'], dtype='object')

In [57]:
fig = px.line(publico_daily_news,y="Número de Notícias Diárias", title="Número de Notícias Público")
fig.update_xaxes(rangeslider_visible=True)
fig.show()